[<< 11. Concurrent programming (Part II)](11_concurrent_programming_part_ii.ipynb) | [Index](00_index.ipynb) | [13. Using an API with Python >>](13_using_an_api.ipynb)

# SQLite - Built-in Python database 

### Creating a DB

In [ ]:
import sqlite3

In [ ]:
def create_connection():
    """ create a database connection to a database that resides
        in the memory
    """
    conn = None
    try:
        conn = sqlite3.connect(':memory:')
        print(sqlite3.version)
    except sqlite3.Error as e:
        print(e)
    finally:
        if conn:
            conn.close()

In [ ]:
create_connection()

In [ ]:
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        return conn
    except sqlite3.Error as e:
        print(e)

    return conn

In [ ]:
database = "11_databases/sample_database.db"
conn = create_connection(database)

### Inserting data

In [ ]:
def create_table(conn, create_table_sql):
    """ create a table from the create_table_sql statement
    :param conn: Connection object
    :param create_table_sql: a CREATE TABLE statement
    :return:
    """
    try:
        c = conn.cursor()
        c.execute(create_table_sql)
    except sqlite3.Error as e:
        print(e)

In [ ]:
sql_create_projects_table = """ CREATE TABLE IF NOT EXISTS projects (
                                id integer PRIMARY KEY,
                                name text NOT NULL,
                                begin_date text,
                                end_date text
                            ); """

sql_create_tasks_table = """CREATE TABLE IF NOT EXISTS tasks (
                            id integer PRIMARY KEY,
                            name text NOT NULL,
                            priority integer,
                            status_id integer NOT NULL,
                            project_id integer NOT NULL,
                            begin_date text NOT NULL,
                            end_date text NOT NULL,
                            FOREIGN KEY (project_id) REFERENCES projects (id)
                        );"""

In [ ]:
# create tables
if conn is not None:
    # create projects table
    create_table(conn, sql_create_projects_table)

    # create tasks table
    create_table(conn, sql_create_tasks_table)
else:
    print("Error! cannot create the database connection.")

In [ ]:
def add_project(conn, project):
    """
    Create a new project into the projects table
    :param conn:
    :param project:
    :return: project id
    """
    sql = ''' INSERT INTO projects(name,begin_date,end_date)
              VALUES(?,?,?) '''
    cur = conn.cursor()
    cur.execute(sql, project)
    conn.commit()
    return cur.lastrowid

In [ ]:
def add_task(conn, task):
    """
    Create a new task
    :param conn:
    :param task:
    :return:
    """

    sql = ''' INSERT INTO tasks(name,priority,status_id,project_id,begin_date,end_date)
              VALUES(?,?,?,?,?,?) '''
    cur = conn.cursor()
    cur.execute(sql, task)
    conn.commit()

    return cur.lastrowid

In [ ]:
database = "11_databases/sample_database.db"

# create a database connection
conn = create_connection(database)
with conn:
    # create a new project
    project = ('Cool App with SQLite & Python', '2015-01-01', '2015-01-30');
    project_id = add_project(conn, project)

    # tasks
    task_1 = ('Analyze the requirements of the app', 1, 1, project_id, '2015-01-01', '2015-01-02')
    task_2 = ('Confirm with user about the top requirements', 1, 1, project_id, '2015-01-03', '2015-01-05')

    # create tasks
    add_task(conn, task_1)
    add_task(conn, task_2)

### Selecting data

In [ ]:
def select_all_tasks(conn):
    """
    Query all rows in the tasks table
    :param conn: the Connection object
    :return:
    """
    cur = conn.cursor()
    cur.execute("SELECT * FROM tasks")

    rows = cur.fetchall()

    for row in rows:
        print(row)

In [ ]:
def select_task_by_priority(conn, priority):
    """
    Query tasks by priority
    :param conn: the Connection object
    :param priority:
    :return:
    """
    cur = conn.cursor()
    cur.execute("SELECT * FROM tasks WHERE priority=?", (priority,))

    rows = cur.fetchall()

    for row in rows:
        print(row)

In [ ]:
database = "11_databases/sample_database.db"

# create a database connection
conn = create_connection(database)
with conn:
        print("1. Query task by priority:")
        select_task_by_priority(conn, 1)

        print("2. Query all tasks")
        select_all_tasks(conn)

### Updating data

In [ ]:
def update_task(conn, task):
    """
    update priority, begin_date, and end date of a task
    :param conn:
    :param task:
    :return: project id
    """
    sql = ''' UPDATE tasks
              SET priority = ? ,
                  begin_date = ? ,
                  end_date = ?
              WHERE id = ?'''
    cur = conn.cursor()
    cur.execute(sql, task)
    conn.commit()

In [ ]:
database = "11_databases/sample_database.db"
# create a database connection
conn = create_connection(database)
with conn:
    update_task(conn, (2, '2015-01-04', '2015-01-06', 2))

### Deleting data

In [ ]:
def delete_task(conn, id):
    """
    Delete a task by task id
    :param conn:  Connection to the SQLite database
    :param id: id of the task
    :return:
    """
    sql = 'DELETE FROM tasks WHERE id=?'
    cur = conn.cursor()
    cur.execute(sql, (id,))
    conn.commit()

In [ ]:
def delete_all_tasks(conn):
    """
    Delete all rows in the tasks table
    :param conn: Connection to the SQLite database
    :return:
    """
    sql = 'DELETE FROM tasks'
    cur = conn.cursor()
    cur.execute(sql)
    conn.commit()

In [ ]:
database = "11_databases/sample_database.db"

conn = create_connection(database)
with conn:
    delete_task(conn, 2)
    # delete_all_tasks(conn)

## 3rd party libraries for SQL database connections

- mysql connector: https://dev.mysql.com/doc/connector-python/en/
- PostgreSQL: https://www.psycopg.org/docs/
- sqlalchemy: https://docs.sqlalchemy.org/en/20/

[<< 11. Concurrent programming (Part II)](11_concurrent_programming_part_ii.ipynb) | [Index](00_index.ipynb) | [13. Using an API with Python >>](13_using_an_api.ipynb)